In [1]:
import torch
import numpy as np
from utils import get_score, normalize
result_dir='/home/dyj/'
sigmoid = torch.sigmoid

In [2]:
label = torch.load('/home/dyj/label.pt')

In [ ]:
# cnn1 = torch.load('/home/dyj/TextCNN1_2017-07-27#10:15:20_res.pt')
# cnn1_loss_weight = torch.load(result_dir+'TextCNN1_loss_weight.pt')

In [ ]:
# rnn1 = torch.load('/home/dyj/RNN1_2017-07-27#10:48:05_res.pt')
# rnn1_loss_weight = torch.load(result_dir+'RNN1_loss_weight.pt')

In [ ]:
# rcnn1 = torch.load(result_dir + 'RCNN1_2017-07-27#11:01:07_res.pt')
# rcnn1_loss_weight = torch.load('/home/dyj/RCNN1_loss_weight.pt')

In [ ]:
# rcnncha = torch.load('/home/dyj/RCNNcha_2017-07-27#16:19:23_res.pt')
# rcnncha_loss_weight = torch.load(result_dir+'RCNNcha_loss_weight.pt')

In [ ]:
# fasttext1_char = torch.load('snapshots/FastText/layer_1_cal_res_char.pt')
# fasttext1_loss_weight = torch.load('snapshots/FastText/layer_2_loss_weight_char.pt')

In [ ]:
# fasttext4 = torch.load(result_dir + 'FastText4_2017-07-28#15:14:47_res.pt')
# fasttext4_loss_weight = torch.load('/home/dyj/FastText4_loss_weight.pt')

In [ ]:
# fasttext10 = torch.load('/home/dyj/FastText10_res.pt')
# fasttext10_loss_weight = torch.load('/home/dyj/FastText10_loss_weight.pt')

In [ ]:
# rnn10_cnn7 = torch.load('snapshots/TextCNN/layer_17_cal_res_3.pt')
# rnn10_cnn7_loss_weight = torch.load('snapshots/TextCNN/layer_18_loss_weight_3.pt')

In [ ]:
# cnn4 = torch.load('snapshots/TextCNN/layer_4_cal_res_3.pt')
# cnn4_loss_weight = torch.load('snapshots/TextCNN/layer_5_loss_weight_3.pt')

In [3]:
def get_loss_weight(logit, label):
    class_num = logit.size(1)
    predict_label_list = [list(ii) for ii in logit.topk(5, 1)[1]]
    marked_label_list = [list(np.where(ii.numpy()==1)[0]) for ii in label]
    sample_per_class = torch.zeros(class_num)
    error_per_class = torch.zeros(class_num)
    for predict_labels, marked_labels in zip(predict_label_list, marked_label_list):
        for true_label in marked_labels:
            sample_per_class[true_label] += 1
            if true_label not in predict_labels:
                error_per_class[true_label] += 1
    return error_per_class / sample_per_class

In [4]:
rnn10 = torch.load(result_dir + 'RNN10_cal_res.pt')
rnn10_loss_weight = torch.load('snapshots/RNN/layer_11_loss_weight_3.pt')

In [5]:
logit1 = sigmoid(rnn10/10) * torch.sqrt(1-rnn10_loss_weight+rnn10_loss_weight.mean()).expand_as(rnn10)
# get_score(logit1, label)
# 0.42681

In [ ]:
# rnn10_cnn5 = torch.load('snapshots/TextCNN/layer_15_cal_res_char.pt')
# cnn5_char = rnn10_cnn5 - rnn10
# cnn5_char = torch.load(result_dir + 'TextCNN5_char_top5.pt')
# cnn5_char_loss_weight = get_loss_weight(cnn5_char, label)

In [ ]:
# logit2 = sigmoid(cnn5_char/5) * torch.sqrt(1-cnn5_char_loss_weight+cnn5_char_loss_weight.mean()).expand_as(cnn5_char)
# get_score(logit2, label)
# 0.41667

In [6]:
cnn10_char = torch.load('snapshots/TextCNN/layer_10_cal_res_char.pt')
cnn10_char_loss_weight = get_loss_weight(cnn10_char, label)

In [7]:
logit2 = sigmoid(cnn10_char/10) * torch.sqrt(1-cnn10_char_loss_weight+cnn10_char_loss_weight.mean()).expand_as(cnn10_char)
# get_score(logit2, label)
# 0.41843

In [ ]:
cnn10_top1 = torch.load('snapshots/TextCNN/layer_10_cal_res_top1.pt')
cnn10_top1_loss_weight = get_loss_weight(cnn10_top1, label)

In [ ]:
logit3 = sigmoid(cnn10_top1/10) * torch.sqrt(1-cnn10_top1_loss_weight+cnn10_top1_loss_weight.mean()).expand_as(cnn10_top1)
# get_score(logit3, label)
# 0.42705

In [ ]:
cnn8_top1_char = torch.load('snapshots/TextCNN/layer_8_cal_res_top1_char.pt')
cnn8_top1_char_loss_weight = get_loss_weight(cnn8_top1_char, label)

In [ ]:
logit4 = sigmoid(cnn8_top1_char/8) * torch.sqrt(1-cnn8_top1_char_loss_weight+cnn8_top1_char_loss_weight.mean()).expand_as(cnn8_top1_char)
# get_score(logit4, label)
# 0.41815

In [ ]:
cnn4 = torch.load(result_dir + 'TextCNN4_cal_res.pt')
cnn4_loss_weight = torch.load('snapshots/TextCNN/layer_5_loss_weight_3.pt')

In [ ]:
logit5 = sigmoid(cnn4/4) * torch.sqrt(1-cnn4_loss_weight+cnn4_loss_weight.mean()).expand_as(cnn4)
# get_score(logit5, label)
# 0.42331

In [ ]:
fasttext10 = torch.load('/home/dyj/FastText10_res.pt')
fasttext10_loss_weight = torch.load('/home/dyj/FastText10_loss_weight.pt')

In [ ]:
logit6 = sigmoid(fasttext10/10) * torch.sqrt(1-fasttext10_loss_weight+fasttext10_loss_weight.mean()).expand_as(fasttext10)
# get_score(logit6, label)
# 0.41932

In [8]:
cnn1_word_shuffle = torch.load('snapshots/TextCNN/layer_1_cal_res_augment.pt')
cnn1_word_shuffle_loss_weight = torch.load('snapshots/TextCNN/layer_2_loss_weight_augment.pt')

In [9]:
logit7 = sigmoid(cnn1_word_shuffle) * torch.sqrt(1-cnn1_word_shuffle_loss_weight+cnn1_word_shuffle_loss_weight.mean()).expand_as(cnn1_word_shuffle)
# get_score(logit7, label)
# 0.41449

(1.4278054949663341, 0.5840333022868975, 0.41448945083824457)

In [ ]:
logit = logit1*0.23+logit2*0.146+logit3*0.16+logit4*0.1423+logit5*0.1024+logit6*0.0183+logit7*0.0695
get_score(logit, label)

In [10]:
logit = logit1*0.32+logit2*0.2
get_score(logit, label)

(1.4854828281018575, 0.6066786864794196, 0.43075583058936806)

In [25]:
logit = logit1*0.31+logit2*0.21+logit7*0.05
get_score(logit, label)

(1.4860091651974103, 0.6070304511417398, 0.43097742006827744)

In [38]:
cnn1_char = torch.load('snapshots/TextCNN/layer_1_cal_res_top1_char.pt')
cnn1_char_loss_weight = get_loss_weight(cnn1_char, label)

In [39]:
logit4_ = sigmoid(cnn1_char) * torch.sqrt(1-cnn1_char_loss_weight+cnn1_char_loss_weight.mean()).expand_as(cnn1_char)
get_score(logit4_, label)
# 0.4032

(1.3904943267159242, 0.5678720658981962, 0.4032048797967101)

In [41]:
rnn1 = torch.load('snapshots/RNN/layer_1_cal_res_3.pt')
rnn1_loss_weight = torch.load('snapshots/pshots/RNN/layer_2_loss_weight_3.pt')

In [43]:
logit1_ = sigmoid(rnn1) * torch.sqrt(1-rnn1_loss_weight+rnn1_loss_weight.mean()).expand_as(rnn1)
# get_score(logit1_, label)
# 0.41223

(1.4232794130377397, 0.5803105782897117, 0.41223209480126921)

In [49]:
# rnn1 + cnn1_char
logit = logit1_*1.2 + logit4_
get_score(logit, label)
# 0.42170

(1.4553865326191719, 0.5937303289498045, 0.42169733750541838)

In [55]:
# rnn1 + cnn1_augment
logit = logit1_ + logit7*1.2
get_score(logit, label)
# 0.42103

(1.4521791365583794, 0.5929328058287547, 0.421025681851182)

In [64]:
# rnn10 + cnn1_char
logit = logit1*4 + logit4_ 
get_score(logit, label)
# 0.42922

(1.4804827055125227, 0.6044712564122283, 0.42922254277096034)

In [71]:
# rnn10 + cnn1_augment
logit = logit1*5 + logit7
get_score(logit, label)
# 0.42736x

(1.4737875751506517, 0.6018906994562602, 0.42735863515522943)

In [ ]:
%cd utils
from resmat import search_model_weight

In [ ]:
models = [logit1, logit2, logit3, logit4, logit5]
logit = torch.stack(models, 2)

In [ ]:
init_weight = torch.Tensor([0.4, 0.18, 0.22, 0.1, 0.1])

In [ ]:
cur_weight = torch.load('../snapshots/Stack/model_weight.pt')
cur_weight

In [ ]:
search_model_weight(logit, label, init_weight, 10, 0.001, '../snapshots/Stack/model_weight.pt')

In [ ]:
layer_11 = torch.load('snapshots/TextCNN/layer_11_cal_res_char.pt')
layer_12 = torch.load('snapshots/TextCNN/layer_12_cal_res_char.pt')

In [ ]:
layer_12_weighted = (layer_11 / 11 * 5.5 + (layer_12-layer_11)) / 6.5 * 12
layer_12_loss_weight = get_loss_weight(layer_12_weighted, label)

In [ ]:
print get_score(layer_12_weighted, label)
get_score(sigmoid(layer_12_weighted/12) * torch.sqrt(1-layer_12_loss_weight+layer_12_loss_weight.mean()).expand_as(layer_12_weighted), label)

In [ ]:
torch.save(layer_12_weighted, 'snapshots/TextCNN/layer_12_weighted_cal_res_char.pt')
torch.save(layer_12_loss_weight, 'snapshots/TextCNN/layer_13_weighted_loss_weight_char.pt')

In [ ]:
rnn_cnn2_test = torch.load('results/RNN10_CNN2_test_res.pt')
cnn3_test = torch.load('results/CNN3_2017-08-07#12:41:45_test_res.pt')
cnn2_test = torch.load('results/CNN2_2017-08-07#12:53:50_test_res.pt')
torch.save(rnn_cnn2_test+cnn3_test+cnn2_test, '/home/dyj/RNN10_CNN7_test_res.pt')
torch.save(rnn_cnn2_test+cnn3_test+cnn2_test, 'results/RNN10_CNN7_test_res.pt')

In [ ]:
logit = sigmoid(cnn1) * torch.sqrt(1-cnn1_loss_weight+cnn1_loss_weight.mean()).expand_as(cnn1) * 0 + \
        sigmoid(rnn1) * torch.sqrt(1-rnn1_loss_weight+rnn1_loss_weight.mean()).expand_as(rnn1) * 0. + \
        sigmoid(rcnn1) * torch.sqrt(1-rcnn1_loss_weight+rcnn1_loss_weight.mean()).expand_as(rcnn1) * 0 + \
        sigmoid(rcnncha) * torch.sqrt(1-rcnncha_loss_weight+rcnncha_loss_weight.mean()).expand_as(rcnncha) * 0 + \
        sigmoid(fasttext1_char) * torch.sqrt(1-fasttext1_loss_weight+fasttext1_loss_weight.mean()).expand_as(fasttext1_char) * 0 + \
        sigmoid(fasttext4/4) * torch.sqrt(1-fasttext4_loss_weight+fasttext4_loss_weight.mean()).expand_as(fasttext4) * 0 + \
        sigmoid(fasttext10/10) * torch.sqrt(1-fasttext10_loss_weight+fasttext10_loss_weight.mean()).expand_as(fasttext10) * 0.0469 + \
        sigmoid(rnn10_cnn7/17) * torch.sqrt(1-rnn10_cnn7_loss_weight+rnn10_cnn7_loss_weight.mean()).expand_as(rnn10_cnn7) * 0.6604 + \
        sigmoid(cnn4/4) * torch.sqrt(1-cnn4_loss_weight+cnn4_loss_weight.mean()).expand_as(cnn4) * 0.1536

In [ ]:
get_loss_weight??
rnn10_cnn8_loss_weight_new = get_loss_weight(rnn10_cnn8, label)

In [ ]:
torch.save(rnn10_cnn8_loss_weight_new, 'snapshots/TextCNN/layer_19_loss_weight_top1.pt')

In [ ]:
layer_18_loss_weight_top3 = torch.load('snapshots/TextCNN/layer_18_loss_weight_top3.pt')
layer_19_loss_weight_top3 = torch.load('snapshots/TextCNN/layer_19_loss_weight_top3.pt')

In [ ]:
print layer_18_loss_weight_top3.mean()
layer_19_loss_weight_top3.mean()